In [2]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1. MovieLens 데이터 셋 불러오기

In [3]:
path = "/content/drive/MyDrive/Movie_recommend/movielens"

In [4]:
ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding = 'utf-8' )
movies_df = pd.read_csv(os.path.join(path, 'movies.csv'), index_col = 'movieId',encoding = 'utf-8' )
tags_df = pd.read_csv(os.path.join(path, 'tags.csv'), encoding = 'utf-8' )

In [7]:
print(ratings_df)
print(ratings_df.shape)
print(ratings_df.info())
ratings_df

        userId  movieId  rating   timestamp
0            1        1     4.0   964982703
1            1        3     4.0   964981247
2            1        6     4.0   964982224
3            1       47     5.0   964983815
4            1       50     5.0   964982931
...        ...      ...     ...         ...
100831     610   166534     4.0  1493848402
100832     610   168248     5.0  1493850091
100833     610   168250     5.0  1494273047
100834     610   168252     5.0  1493846352
100835     610   170875     3.0  1493846415

[100836 rows x 4 columns]
(100836, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [13]:
len(set(ratings_df['userId']))

610

In [15]:
num_users = ratings_df['userId'].unique()
num_movies = ratings_df['movieId'].unique()

print(len(num_users), len(num_movies))

610 9724


In [16]:
user_movie_matrix = ratings_df.pivot(index = 'movieId', columns = 'userId', values = 'rating').fillna(0)
user_movie_matrix

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,4.5,3.5,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.5,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,2.5,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
movie_info_df = pd.DataFrame(data=[sum(list(user_movie_matrix[int(x)].value_counts())[1:]) for x in user_movie_matrix.columns], 
             index = user_movie_matrix.columns, columns = ['movies_rated'])
movie_info_df

,movies_rated
userId,
1,232
2,29
3,39
4,216
5,44
...,...
606,1115
607,187
608,831


In [31]:
user_info_df = pd.DataFrame(data=[sum(list(user_movie_matrix.loc[int(x)].value_counts())[1:]) for x in user_movie_matrix.index], 
             index = user_movie_matrix.index, columns = ['users_rated'])
user_info_df

,users_rated
movieId,
1,215
2,110
3,52
4,7
5,49
...,...
193581,1
193583,1
193585,1


In [33]:
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=1234)

In [69]:
train_df.index

Int64Index([95713, 61560, 77204, 93367, 90892, 89897, 49995, 65192, 32532,
            89574,
            ...
             9449, 41239, 55985, 32399, 82584, 89460, 60620, 34086, 58067,
            92975],
           dtype='int64', length=80668)

In [34]:
print(train_df.shape, test_df.shape)

(80668, 4) (20168, 4)


test set에는 존재하지만, train set에는 없는 영화 또는 사용자 비율

In [36]:
len(list(set(test_df['userId'].unique()) - set(train_df['userId'].unique())))

0

In [37]:
len(list(set(test_df['movieId'].unique()) - set(train_df['movieId'].unique())))

786

In [39]:
print(f"사용자: {len(list(set(test_df['userId'].unique()) - set(train_df['userId'].unique())))}")
print(f"사용자: {len(list(set(test_df['movieId'].unique()) - set(train_df['movieId'].unique())))}")
print(f"테스트셋의 전체 영화수: {len(test_df['movieId'].unique())}")

사용자: 0
사용자: 786
테스트셋의 전체 영화수: 5171


train set에는 존재하지만, test set에는 없는 영화 또는 사용자 비율

In [41]:
print(f"사용자: {len(list(set(train_df['userId'].unique()) - set(test_df['userId'].unique())))}")
print(f"사용자: {len(list(set(train_df['movieId'].unique()) - set(test_df['movieId'].unique())))}")
print(f"트레인셋의 전체 영화수: {len(train_df['movieId'].unique())}")

사용자: 0
사용자: 4553
트레인셋의 전체 영화수: 8938


In [43]:
print(len(train_df['movieId']))
print(len(test_df['movieId']))

80668
20168


## 2. 간단한 추천 알고리즘 만들기

##### 1. 랜덤으로 평점예측

In [50]:
ratings_range = np.arange(0.5, 5.5, step = 0.5)

In [52]:
import random
pred_random = [random.choice(ratings_range) for x in range(len(test_df))]
pred_random

[1.5,
 4.0,
 3.5,
 3.5,
 4.5,
 4.0,
 0.5,
 4.0,
 2.0,
 4.0,
 4.0,
 1.5,
 3.5,
 5.0,
 3.0,
 3.5,
 3.5,
 2.5,
 3.0,
 3.5,
 1.5,
 1.0,
 4.0,
 1.0,
 3.0,
 2.5,
 1.0,
 0.5,
 2.0,
 3.0,
 3.0,
 3.0,
 5.0,
 1.0,
 1.5,
 3.5,
 5.0,
 2.0,
 3.0,
 4.5,
 3.0,
 1.5,
 5.0,
 3.0,
 0.5,
 5.0,
 4.5,
 5.0,
 1.5,
 5.0,
 1.5,
 2.0,
 3.0,
 1.5,
 3.0,
 1.0,
 3.5,
 4.0,
 3.5,
 1.0,
 2.5,
 4.5,
 2.5,
 2.5,
 4.0,
 1.5,
 4.0,
 2.0,
 3.5,
 1.5,
 3.5,
 0.5,
 3.0,
 4.0,
 5.0,
 3.5,
 3.5,
 1.0,
 0.5,
 3.0,
 2.5,
 4.5,
 2.5,
 0.5,
 3.5,
 4.0,
 2.0,
 2.0,
 2.5,
 4.5,
 3.0,
 2.5,
 4.5,
 1.5,
 0.5,
 4.0,
 2.0,
 5.0,
 1.5,
 3.5,
 5.0,
 0.5,
 3.5,
 0.5,
 0.5,
 3.5,
 0.5,
 1.0,
 4.5,
 1.0,
 2.0,
 2.5,
 3.0,
 5.0,
 2.0,
 0.5,
 2.5,
 3.5,
 3.5,
 2.0,
 2.5,
 0.5,
 5.0,
 1.0,
 1.5,
 3.5,
 0.5,
 1.5,
 1.5,
 5.0,
 5.0,
 2.5,
 4.0,
 0.5,
 3.0,
 5.0,
 3.5,
 2.0,
 1.0,
 0.5,
 5.0,
 4.5,
 1.0,
 4.0,
 2.5,
 4.0,
 1.0,
 1.5,
 0.5,
 0.5,
 5.0,
 1.5,
 2.0,
 3.5,
 1.5,
 3.5,
 0.5,
 0.5,
 2.5,
 1.5,
 1.5,
 2.0,
 2.5,
 3.0,
 5.0,
 3.0,
 3.5

In [53]:
test_df['pred_ratings_random'] = pred_random
test_df

,userId,movieId,rating,timestamp,pred_ratings_random
99731,610,3527,5.0,1479545223,1.5
97583,606,1250,3.5,1171376891,4.0
38197,262,213,5.0,840310907,3.5
11474,68,69406,3.0,1261622505,3.5
34105,232,4728,3.0,1218166950,4.5
...,...,...,...,...,...
41080,279,593,4.0,1506394242,1.0
4897,31,780,4.0,850466616,1.5
8023,56,410,3.0,835799188,4.5
77467,483,2291,4.0,1415579167,3.5


In [54]:
mse = mean_squared_error(test_df['rating'].values, test_df['pred_ratings_random'].values)
rmse = np.sqrt(mse)

print(mse, rmse)

3.6970076358587862 1.922760420816589


#### 2. 사용자 평균 평점을 기반으로 예측

In [55]:
train_user_df = train_df.groupby('userId').mean()

print(train_user_df.shape)
print(train_user_df.head())

(610, 3)
             movieId    rating     timestamp
userId                                      
1        1891.168478  4.320652  9.649865e+08
2       70402.760000  3.940000  1.445715e+09
3        8394.733333  2.516667  1.306464e+09
4        1957.923077  3.631868  9.655941e+08
5         337.606061  3.636364  8.474351e+08


In [59]:
def avg_rating_prediction(train_set, x): #????
  if x in train_set.index:
    pred_rating = train_set.loc[x]['rating']
  else:
    pred_rating = random.choice(ratings_range)
  return pred_rating

In [60]:
test_df['pred_rating_user'] = test_df['userId'].apply(lambda x: avg_rating_prediction(train_user_df, x))

test_df

,userId,movieId,rating,timestamp,pred_ratings_random,pred_rating_user
99731,610,3527,5.0,1479545223,1.5,3.678709
97583,606,1250,3.5,1171376891,4.0,3.649718
38197,262,213,5.0,840310907,3.5,2.925000
11474,68,69406,3.0,1261622505,3.5,3.229331
34105,232,4728,3.0,1218166950,4.5,3.242268
...,...,...,...,...,...,...
41080,279,593,4.0,1506394242,1.0,3.666667
4897,31,780,4.0,850466616,1.5,3.911765
8023,56,410,3.0,835799188,4.5,3.837838
77467,483,2291,4.0,1415579167,3.5,3.598940


#### 3. 영화 평균 평점을 기반으로 예측

In [62]:
train_movie_df = train_df.groupby('movieId').mean()
test_df['pred_rating_movie'] = test_df['movieId'].apply(lambda x: avg_rating_prediction(train_movie_df, x))

test_df

,userId,movieId,rating,timestamp,pred_ratings_random,pred_rating_user,pred_rating_movie
99731,610,3527,5.0,1479545223,1.5,3.678709,3.604167
97583,606,1250,3.5,1171376891,4.0,3.649718,4.180556
38197,262,213,5.0,840310907,3.5,2.925000,3.750000
11474,68,69406,3.0,1261622505,3.5,3.229331,3.571429
34105,232,4728,3.0,1218166950,4.5,3.242268,2.769231
...,...,...,...,...,...,...,...
41080,279,593,4.0,1506394242,1.0,3.666667,4.127907
4897,31,780,4.0,850466616,1.5,3.911765,3.470760
8023,56,410,3.0,835799188,4.5,3.837838,3.131148
77467,483,2291,4.0,1415579167,3.5,3.598940,3.734375


In [67]:
mse_user = mean_squared_error(test_df['rating'].values, test_df['pred_rating_user'].values)
rmse_user = np.sqrt(mse_user)

print(mse_user, rmse_user)

0.8905889036428333 0.9437101798978504


In [68]:
mse_movie = mean_squared_error(test_df['rating'].values, test_df['pred_rating_movie'].values)
rmse_movie = np.sqrt(mse_movie)

print(mse, rmse_movie)

1.0668462090154196 1.0328824758971467


### ** word2vec **

## 